In [ ]:
import os,json,csv,asyncio,requests,time

In [ ]:
import requests,asyncio,json,csv, os,aiohttp
from dotenv import load_dotenv

load_dotenv()
MAL_CLIENT_ID = os.getenv("MAL_CLIENT_ID")

In [ ]:
with open("./data/users.csv","w",newline="") as csvfile:
    writer = csv.writer(csvfile)
    writer.writerow(["user","manga_id","status","is_rereading","score","chapters_read","updated_at"])
    for file in os.listdir("./user"):
        
        with open("./user/"+file) as f:
            try:
                data = json.load(f)
            except:
                continue
            #data = json.load(f)
            #print(file)
            for item in data.get("data",[]):
                try:
                    writer.writerow([file[:-5],item["node"]["id"],item["list_status"]["status"],item["list_status"]["is_rereading"],item["list_status"]["score"],item["list_status"]["num_chapters_read"],item["list_status"]["updated_at"]])
                except:
                    pass


In [ ]:
import pandas as pd

In [ ]:
df = pd.read_csv("./data/users.csv")


In [ ]:
df["manga_id"].value_counts().to_csv("./data/manga_id.csv")

In [ ]:
# dont include manga with no score
df2 = df[df["score"]!=0]


In [ ]:
def get_content_data(content_id:str,content_type:str="manga"):
    if content_type not in ["manga","anime"]:
        raise ValueError("content_type must be either 'manga' or 'anime'")
    response = requests.get(f"https://api.myanimelist.net/v2/{content_type}/{content_id}?fields=[genres,authors,background,start_date,status,popularity,rank,synopsis,mean,num_list_users,num_scoring_users,nsfw,num_chapters]", headers={"X-MAL-CLIENT-ID":MAL_CLIENT_ID})
    print("got response for",content_id)
    with open(f"./data/{content_type}/{content_id}.json","w") as f:
        json.dump(json.loads(response.text),f)

In [ ]:
mangas = set()
for manga in os.listdir("./data/manga"):
    mangas.add(int(manga[:-5]))
mangas_to_get = set(df2["manga_id"].value_counts().index)

In [ ]:
mangas_to_get = mangas_to_get.difference(mangas)

In [ ]:
tasks = []
for manga_id in mangas_to_get:
    time.sleep(1)
    tasks.append(get_content_data(manga_id))
    

In [ ]:
with open("mangas.csv","w",newline="",encoding="utf-8") as manga:
    writer = csv.writer(manga)
    writer.writerow(["manga_id","title","genres","authors","background","start_date","status","popularity","rank","synopsis","mean","num_list_users","num_scoring_users","nsfw","num_chapters"])
    for manga in os.listdir("./data/manga"):
        with open(f"./data/manga/{manga}",encoding="utf-8") as f:
            try:
                data:dict = json.load(f)
            except:
                print(f"failed to load {manga}")
                continue
            try:
                writer.writerow([data["id"],data["title"],[genre["name"] for genre in data.get("genres",[])],[(author["node"]["id"],author[ "role"]) for author in data.get("authors",[])],data["background"],data.get("start_date",None),data["status"],data["popularity"],data.get("rank"),data["synopsis"],data.get("mean",None),data["num_list_users"],data["num_scoring_users"],data["nsfw"],data["num_chapters"]])
            except Exception as e:  
                print(e)

In [ ]:
from recomendations.recomendator import Recomendator

In [ ]:
rec = Recomendator("users.csv","mangas.csv")

In [ ]:
rec.get_recommendations("Jseph22")

In [ ]:
rec.mangas[rec.mangas["title"]=="Bleach"]

In [ ]:
rec.svd.predict(uid=rec.user_to_uid.get("TheMissingTrex"), iid=rec.manga_to_iid.get("Steel-Eating Player")).est